Instalacja pakietu pyjanitor:

In [ ]:
%pip install pyjanitor

---

In [ ]:
# załadowanie bibliotek

import pandas as pd
import numpy as np
import janitor

In [ ]:
# wczytanie danych

dane = pd.ExcelFile('data.xlsx').parse('pracownicy', index_col = 0)                    

In [ ]:
# zanieczyszczenie danych

import warnings

warnings.simplefilter(category = FutureWarning, action = 'ignore')

dane_brudne = dane.rename(columns = {'Stanowisko':'Zawód'}
                 ).assign(**{'Imię'                 : lambda df: df['Imię'].where(df.index != 'AN01', 'Alex'),
                             'Płeć'                 : ['kobieta', 'mężczyzna', 'inna', 'kobieta', 'kobieta', 'mężczyzna', 'mężczyzna', 'kobieta', 'mężczyzna', 'mężczyzna', 'mężczyzna', 'mężczyzna'],
                             'Data urodzenia'       : lambda df: df['Data urodzenia'].astype(str),
                             'Zarobki'              : lambda df: df.index.map(lambda x: 3100000   if x == 'BS99' else (
                                                                                        '4400 zł' if x == 'LW89' else df.loc[x, 'Zarobki'])),
                             'Zawód'                : lambda df: df.index.map(lambda x: 'analityk  danych' if x == 'KM00' else (
                                                                                        'Analityk'         if x == 'AN01' else (
                                                                                        'analtyk'          if x == 'ŁG96' else (
                                                                                        'analityk danych'  if x == 'AP98' else (
                                                                                        'programista '     if x == 'AK94' else df.loc[x, 'Zawód']))))),
                             'Multisport'           : ['Tak', 'Tak', 'Nie', 'Nie', '-', 'Tak', 'Nie', 'Tak', 'Tak', 'Nie wiem', 'Nie', 'Tak'], 
                             'Hobby'                : ['Sport', 'Nauka', None, 'Inne: Sztuka', 'Sport', 'Podróże', 'Inne: Wędkarstwo', 'Inne: Historia', 'Nauka', 'Inne: kultura', None, 'Podróże'],
                             'Prawie pusta kolumna' : [None, None, 1490, None, 980, None, None, None, None, 3, None, None],
                             'Pusta kolumna'        : None,
                             'Jedna wartość'        : -7,
                             }
                 ).pipe(lambda df: df.insert(dane.columns.get_loc('Nazwisko') + 1, 'Płeć', df.pop('Płeć')) or df
                 ).pipe(lambda df: pd.concat([df.loc[:'AP98'],
                                              df.loc['JM00':'AP98'],
                                              df.loc['WN88':],
                                              pd.DataFrame([np.hstack([[np.nan] * 10, None, np.nan])], index = ['AB12'], columns = df.columns, dtype = 'object')])
                 )

dane = dane_brudne.copy()

---

# Czyszczenie danych

**czyszczenie danych** (ang. *data cleaning*) - przygotowanie danych do przetworzenia i analizy danych, ich uporządkowanie i uspójnienie

**Elementy procesu czyszczenia danych**:

1. Uporządkowanie zbioru kolumn

- czyszczenie nazw
    - oczyszczenie stringów
    - zmiana niejasnych i niespójnych nazw
    - dodanie prefiksów do nazw pokrewnych kolumn
    
- zmiana kolejności kolumn

2. Usunięcie zbędnych wierszy i kolumn

- usunięcie pustych wierszy
- usunięcie pustych kolumn
- usunięcie kolumn z jedną wartością
- usunięcie wierszy duplikatów

3. Poprawa poszczególnych wartości

- usunięcie błędów w danych 
- usunięcie braków w danych 

4. Poprawa typów kolumn



5.* Przekształcenie danych do postaci tidy


Najważniejszymi spośród nich są usunięcie błędnych danych, zmiana błędnych typów kolumn i usunięcie wierszy duplikatów, ponieważ często bez ich wykonania analiza nie jest możliwa.

**Uwaga!** Nie wszystkie kroki można wykonać zawsze w tej samej kolejności, czasem trzeba wykonywać je naprzemiennie, powtarzając niektóre z nich.

# 0. Wstępna inspekcja danych

In [ ]:
dane.head()

In [ ]:
dane.info()

## 1. Uporządkowanie zbioru kolumn

In [ ]:
# nazwy i kolejność kolumn

dane.columns

### Czyszczenie nazw

Do procesu czyszczenia nazw należy zwykle m.in.:
- zamiana liter na małe
- zastąpienie spacji znakiem "_"
- usunięcie specyficznych znaków, jak polskie czy specjalne - pozostawienie samych liter, cyfr, "_" i "."
- zmiana niejasnych lub niespójnych nazw na bardziej jasne lub spójne
- w przypadku kolumn należących do jakichś grup, dodanie do ich nazw prefiksu

**Oczyszczenie stringów** - usunięcie wielkich litery, spacji, polskich znaków **:**

In [ ]:
# gdyby nie było pakietu janitor

dane.rename(columns = dict(zip(dane.columns, dane.columns.str.strip(
                                                        ).str.lower(
                                                        ).str.replace({' ':'_', 'ł':'l'}
                                                        ).str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8'))))

In [ ]:
# z pakietem janitor

dane = dane.rename(columns = lambda x: x.strip(                      # usunięcie białych znaków (np. spacje) z początków i końców stringów
                                       ).replace('ł', 'l'))          # jeśli polskie znaki, konieczne, bo poniższa funkcja nie usuwa "ł" 

dane = dane.clean_names()                                            # funkcja czyszcząca nazwy z pakietu janitor
dane

**Zmiana niejasnych i niespójnych nazw :**

In [ ]:
dane.rename(columns = {'zarobki'   :'pensja',
                       'zawod'     :'stanowisko',
                       'multisport':'czy_multisport'},
            inplace = True)
dane

**Dodanie prefiksów do nazw pokrewnych kolumn :**

In [ ]:
cols_with_prefix = dane.loc[:, 'imie':'data_urodzenia'].columns
prefix = 'dane_osobiste.'

dane.rename(columns = lambda x: prefix + x if x in cols_with_prefix else x,
            #inplace = True
           )

### Zmiana kolejności kolumn

In [ ]:
dane = dane.filter(['nazwisko',
                    'imie',
                    'plec',
                    'data_urodzenia',
                    'stanowisko',
                    'pensja',
                    'premia',
                    'czy_multisport',
                    'hobby',
                    'jedna_wartosc',
                    'prawie_pusta_kolumna',
                    'pusta_kolumna'])
dane

In [ ]:
# z grupą kolumn z prefiksem

dane.rename(columns = lambda x: prefix + x if x in cols_with_prefix else x
   ).filter(np.hstack(['pensja',
                       prefix + cols_with_prefix,   # umieszczenie grupy kolumn
                       'stanowisko',
                       'premia']).tolist())

## 2. Usunięcie zbędnych wierszy i kolumn

Do zbędnych kolumn należą:
- puste kolumny
- kolumny z jedną wartością

In [ ]:
# liczba różnych wartości w kolumnach

dane.nunique()

Do zbędnych wierszy należą:
- puste wiersze
- wiersze duplikaty

In [ ]:
# liczba różnych wartości w wierszach

dane.nunique(axis = 'columns')

### Usunięcie pustych wierszy i kolumn

In [ ]:
# gdy mamy domyślne indeksy

dane.remove_empty()

In [ ]:
# gdy mamy zdefiniowane indeksy

dane.dropna(axis = 'index',   how = 'all', inplace = True)
dane.dropna(axis = 'columns', how = 'all', inplace = True)
dane

### Usunięcie kolumn z jedną wartością

In [ ]:
def remove_constant(df):
    return df.loc[:, (df != df.loc[df.index[0]]).any()]

dane = remove_constant(dane)
dane

### Usunięcie wierszy duplikatów

In [ ]:
# czy wiersze są duplikatami

dane.duplicated()

In [ ]:
dane = dane.drop_duplicates()
dane

## 3. Poprawa poszczególnych wartości

W zanieczyszczonych danych trzeba zwykle pozbyć się wartości błędnych i brakujących.

### Obsługa braków w danych 

In [ ]:
# liczba braków danych w kolumnach

len(dane) - dane.count()

**1. Ujednolicenie**

pd.DataFrame często zawiera "NumPy-owy" brak danych - np.NaN (Not a Number), ale czasem zawiera też "Pythonowy" - None. Należy je ujednolicić do np.NaN.

In [ ]:
dane = dane.replace({None: np.nan})
dane

**2a. Usunięcie**

- **Usunięcie wierszy z co najmniej 1 brakiem**

In [ ]:
dane.dropna()

- **Usunięcie kolumn z za dużą liczbą braków**

In [ ]:
thresh_percent = 0.3        # co najmniej taki odsetek ma być niepustych wartości w kolumnie

dane.dropna(thresh = thresh_percent * len(dane), axis = 'columns',
            inplace = True
           )
dane

**2b. Imputacja (uzupełnienie)**

In [ ]:
dane = dane.impute(['hobby'], '')
dane

Metod imputacji brakujących danych wg określonych algorytmów jest wiele. Pozwala to na nieutracenie pozostałych wartości przy niedużej modyfikacji zebranych danych.

### Usunięcie błędnych danych

Aby wyłapać i usunąć błędne dane, często trzeba przeanalizować wartości każdej z kolumn. Pomocna jest przy tym tabela częstości danej zmiennej (funkcja value_counts).

**Nazwisko**

In [ ]:
dane['nazwisko'].value_counts(dropna = False)

**Imię**

In [ ]:
dane['imie'].value_counts(dropna = False)

**Płeć**

In [ ]:
dane['plec'].value_counts(dropna = False)

In [ ]:
dane

In [ ]:
dane['plec'] = dane['plec'].str[0].str.upper()    # zakodowanie zmiennej nominalnej
dane['plec'].value_counts(dropna = False)

**Data urodzenia**

In [ ]:
dane['data_urodzenia'].value_counts(dropna = False)

**Stanowisko**

In [ ]:
dane['stanowisko'].value_counts(dropna = False)

In [ ]:
dane['stanowisko'].unique()                         # różne wartości zmiennej

In [ ]:
dane['stanowisko'] = dane['stanowisko'].str.lower(                                                                                    # małe litery
                                      ).str.strip(                                                                                    # usunięcie białych znaków na początku i końcu stringów
                                      ).mask(dane['stanowisko'].isin(['analityk  danych', 'analityk danych', 'analtyk']), 'analityk') # zmiana błędnych wartości na poprawną (można też z replace)
dane['stanowisko'].value_counts(dropna = False)

**Pensja**

In [ ]:
dane['pensja'].value_counts(dropna = False)

In [ ]:
dane['pensja'] = dane['pensja'].astype(str).str.replace(r'\D', '', regex = True)    # usunięcie znaków, które nie są cyfrą (wyrażenie regularne: \D - znaki inne niż cyfry (\d - cyfry (digit))

In [ ]:
dane['pensja'].astype(int).describe().round(2)                                      # główne statystyki opisowe do wykrycia obserwacji odstających

In [ ]:
dane['pensja'] = dane['pensja'].mask(dane['pensja'].astype(float) > 50000, np.nan)  # zmiana wartości odstających (> 50000) na brak w danych 
dane['pensja'].astype(float).describe()

In [ ]:
dane['pensja'].astype(float).value_counts(dropna = False)

**Premia**

In [ ]:
dane['premia'].value_counts(dropna = False)

In [ ]:
dane['premia'].astype(int).describe().round(2)

**Czy multisport**

In [ ]:
dane['czy_multisport'].value_counts(dropna = False)

In [ ]:
dane['czy_multisport'] = dane['czy_multisport'].replace({'Tak'      : 1,
                                                         'Nie'      : 0,
                                                         '-'        : 0,
                                                         'Nie wiem' : np.nan})
dane['czy_multisport'].value_counts(dropna = False)

**Hobby**

In [ ]:
dane['hobby'].value_counts(dropna = False)

In [ ]:
dane['hobby'] = dane['hobby'].str.lower(
                            ).str.strip(
                            ).replace({'inne: sztuka'     : 'kultura',
                                       'inne: kultura'    : 'kultura',
                                       'inne: wędkarstwo' : 'inne',
                                       'inne: historia'   : 'inne'})
dane['hobby'].value_counts(dropna = False) 

## 4. Poprawa typów zmiennych

Typy kolumn powinny być dopasowane do typu zmiennych jaki reprezentują, aby posiadać funkcjonalności, które posiadają zmienne na danej skali.

In [ ]:
# typy kolumn

dane.dtypes

### Zmiana typu na str, int, float, bool 
- na zmienne nominalne, ilościowe, zero-jedynkowe

In [ ]:
dane['czy_multisport'].astype(str)

In [ ]:
dane['pensja'].astype(float)

In [ ]:
dane['premia'].astype(int)

In [ ]:
dane['czy_multisport'].astype(bool)

### Zmiana typu na category
- na zmienną porządkową

In [ ]:
dane['stanowisko'].astype('category'
                 ).cat.reorder_categories(['sprzedawca', 'analityk', 'programista', 'kierownik'], ordered = True)

### Zmiana typu na datetime
- na zmienną czasową

In [ ]:
pd.to_datetime(dane['data_urodzenia'], format = '%Y-%m-%d')

### Zmiana typów kilku zmiennych

In [ ]:
dane.astype({'data_urodzenia' : 'datetime64[ns]',
             'stanowisko'     : 'category',
             'pensja'         : 'float',
             'premia'         : 'float',
             'czy_multisport' : 'float'})

In [ ]:
dane = dane.astype({'pensja'         : 'float',
                    'premia'         : 'float',
                    'czy_multisport' : 'float'}
          ).to_datetime('data_urodzenia', format = '%Y-%m-%d'
          ).assign(stanowisko = dane['stanowisko'].astype('category').cat.reorder_categories(['sprzedawca', 'analityk', 'programista', 'kierownik'], ordered = True))
dane

In [ ]:
dane.info()

## 5. Przekształcenie danych do postaci tidy

W większości analiz, dane powinny być zapisane w tzw. postaci **tidy**, w której:
 
1. jedna zmienna = jedna kolumna
2. jedna obserwacja = jeden wiersz
3. jedna wartość = jedna komórka
 
Jeśli dane są w innej postaci, możemy osiągnąć spełnianie powyższych warunków przez przekształcenie danych funkcjami takimi jak: **melt**, **pivot**, **transpose**, **stack**, **unstack**, **str.split**, **explode**, **str.cat**, **agg**. \
Zastosowanie pierwszych trzech z nich jest opisane w pliku pandas_1.

In [ ]:
dane

---

## 6. Podziwianie efektów

Od tego zaczęliśmy:

In [ ]:
dane_brudne

a na tym skończyliśmy:

In [ ]:
dane

---

## Ćwiczenie

Wyczyść poniższe dane:

In [ ]:
pd.ExcelFile('data_melt.xlsx').parse('messy_data', header = 1)